# Description

This code performs "Phase 0" of the PFFP data processing automation. It performs the following tasks:

* reads all files in a folder of BD deployments
* identifies if a BD file contains a drop or not
* adds all files containing drops to a separate "Drops Only" folder
* creates a table of the files with drops, the time of file creation, and the number of drops in the file
* exports the above table to the "Drops Only" folder

The required user inputs are as follows:

* file path to folder containing all .bin files
* file path to the "Drops Only" folder
* bluedrop used and associated calibration factors (maybe find a way out of this?)


Import packages

In [15]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from IPython.display import clear_output
import time

User Inputs

In [16]:
# SETUP VARIABLES - USER INPUTS
BD = 3 #Bluedrop file is from 

# paste the filepath to the folder where the BD data is stored
binFolderPath = 'C:/Users/elise/Desktop/BD3 3.16.23 - All Files'

#paste the filepath to an excel file that the analysis results will be printed in
outputPath = 'C:/Users/elise/Desktop/Test data - BD3 3.16.23 -Drops Only' #  Path to pre-existing Folder

Define Functions

In [38]:
#filename = "blog01C5.bin"

# read file, create dataframe, calibrate, filter pore pressure
def readfile(filename):
    global df
    global Time
    global count

    global g2g
    global g18g 
    global g50g 
    global ppm 
    global g200g 
    global gX55g
    global gY55g 
    global g250g
    global ppm_df
    global ppm_avg
    
    file_path = os.path.join(binFolderPath, filename)  # Create the full file path
    if os.path.isfile(file_path): #for every file in the folder

        file = open(file_path, 'rb')  # read file
        element = file.read(3)  # create a byte list with each element having 3 bytes
        data_array = []

        while element:
            iVAl = int.from_bytes(element, byteorder='big', signed=True) # Convert to signed integer before adding to data array
            data_array.append(iVAl)  # adds the reshaped data from the bd file to the data frame
            element = file.read(3)
            
        file.close()

        np_array = np.array(data_array)  # create numpy array from the list
        np_array = np.reshape(np_array, (-1, 10))  # convert the 1d array to 2d array with 10 cols

    df = pd.DataFrame(np_array) # Creates a Dataframe in pandas from the bd data
    df.columns = ['Count', 'File ID', 'g2g', 'g18g', 'g50g', 'ppm', 'g200g', 'gX55g', 'gY55g', 'g250g'] # names columns
    #print(df)

    #calibrate dataframe

    # APPLY CALIBRATION FACTORS
    Time = (df['Count']-df['Count'].iloc[0]+1)/2000 # gives time in s
    count = df["Count"]-df['Count'].iloc[0]

    if BD == 3:  # calibration factors from March 2023
        g2g = (df['g2g']-33570.1)/1614577.9 #- offset# accelerometers are in g
        g18g = (df['g18g']+13495)/163387.2 #- offset
        g50g = (df['g50g']-238817.4)/63779.5 #- offset
        ppm = ((df['ppm']-139040.1)/20705) * 6.89475729 # converts to kPa
        g200g = (df['g200g'] -262332.4)/38888.7 #- offset
        gX55g = (df['gX55g']-70406)/59754.3
        gY55g = (df['gY55g']-69421.1)/141871.5
        g250g = (df['g250g']-39077.4)/13746.9 #- offset

    if BD == 2: # calibration factors from March 2023
        g2g = (df['g2g']+36597.6)/1637627.3 #- offset# accelerometers are in g
        g18g = (df['g18g']-26185.3)/160297.2 #- offset
        g50g = (df['g50g']-212256.9)/63968.7 #- offset
        ppm = ((df['ppm']+55518.9)/18981.7) * 6.89475729 # converts to kPa
        g200g = (df['g200g']-175499.4)/30583.8 #- offset
        gX55g = (df['gX55g']-53629.9)/68590.9
        gY55g = (df['gY55g']-43694.3)/68280.3
        g250g = (df['g250g']-39619.9)/13545.8 #- offset

    if BD == 1: # calibration factors from March 2023
        g2g = (df['g2g']-43727.6)/1625064 #- offset # accelerometers are in g
        g18g = (df['g18g']-45085.6)/160925.7 #- offset
        g50g = (df['g50g']-173493.4)/63944.6 #- offset
        ppm = ((df['ppm']+31776.1)/20679.7) * 6.89475729 # this is kPa
        g200g = (df['g200g'] -731734.3)/32192.4  #- offset
        gX55g = (df['gX55g'] -68837.9)/52137.3
        gY55g = (df['gY55g']-68015.9)/28074.9
        g250g = (df['g250g']+10580)/13688.7 #- offset

    #Filter pore pressure
    ppm_df = pd.DataFrame(ppm) #create dataframe for pore pressure only
    ppm_df.columns = ['raw'] # recorded pore pressure in kPa
    ppm_df['filtered'] = ppm_df['raw'].rolling(50).mean() #take rolling average over a 50-pt window
    ppm_avg = np.average(ppm) #average ppm over whole file

    ppm_df['difference'] = ppm_df['filtered'] - ppm_avg #value to compare against

# Choose which accelerometer to use 

def accPick(): #this function picks the smallest accelerometer that's not maxed out to perform the integration on
    maxAcc = g250g.max()
    global acc
    if maxAcc < 5: #- offset:
        if g18g.max() < 1.8: #offset:  # does an extra check for the 2g because of noise
            acc = g2g
        else:
            acc = g18g 
    elif maxAcc < 18: #- offset
        acc = g18g
    elif maxAcc < 50: #- offset:
        acc = g50g
    else:
        acc = g250g

#Locate the drops

def locate_drops():
    global peaksArray_trimmed
    global nDrops_trimmed
    global nDrops
    global peaks
    global x

    #signal processing of deceleration data
    x = np.array(acc)  # what accelerometer to get the peaks from
    peaks, _ = find_peaks(x, height = 2, distance=1000, prominence=3)  # finds the largest peaks more than 2g spaced at least 10000 counts apart
    peaksArray = np.array(peaks)  # prints a list of the count where the peaks occur
    #peaksArray = filter(filterpeak, peaksArray)
    #print(peaksArray)
    q = (peaksArray.shape) #gives number of peaks
    nDrops = int(q[0]) #number of drops in the file

    #check that pore pressure is above threshold
    difference = ppm_df["difference"]
    ppm_peaks = []
    for peak in peaksArray:
        peak_ppm_array = []
        for i in range(-150, 150):
            #print(peak+i)
            peak_ppm_array.append(difference[peak+i])
        #print(len(peak_ppm_array))
        peak_ppm_max = max(peak_ppm_array)
        peak_ppm_min = min(peak_ppm_array)
        peak_ppm = max(peak_ppm_max, abs(peak_ppm_min)) 
        ppm_peaks.append(peak_ppm)
        #print(peak_ppm)
        #peak_ppm_array.append[peak_ppm]
    #print(ppm_peaks)
    
    if nDrops == 0:
        nDrops_trimmed = 0
    elif nDrops > 0:
        for n in range (0, nDrops):
            #print(ppm_peaks[n])
            if ppm_peaks[n] < 0.5:
                peaksArray_trimmed = np.delete(peaksArray, n)
                a = (peaksArray_trimmed.shape) #gives number of peaks
                nDrops_trimmed = int(a[0])
            else:
                peaksArray_trimmed = peaksArray
                nDrops_trimmed = nDrops
                #print(nDrops_trimmed)

#Plot deceleration, pore pressure, and identified peaks

def peakplot(x, peaks): # Plot showing peak deceleration
    #%matplotlib ipympl
    fig, (ax1, ax2) = plt.subplots(2)
    
    fig.suptitle(filename)
    ax1.plot(x, color = "k", label = "Deceleration (g)")
    ax1.plot(peaks, x[peaks], "x", color = "red", label = "Not a Deployment")
    ax1.plot(peaks, x[peaks], "x", color = "dodgerblue", label = "Deployment")
    ax1.legend()
    ax2.plot(ppm_df["difference"], color = 'k', label = "Pore Pressure difference (kPa)")
    #peakplot = plt.plot(ppm_df["filtered"], label = "filtered")
    ax2.hlines(0.5, 0, 120000, color = 'k', linestyles="--")
    ax2.hlines(-0.5, 0, 120000, color = 'k',linestyles= '--')
    ax2.legend()

    figpath = outputPath+"/Figures"
    plt.savefig(figpath+"/"+filename.split(".")[0])
    plt.close()


Exporting Functions

In [8]:
#Exporting Functions
def exporttoexcel(): #MODIFY THIS TO CREATE THE DESIRED TABLE
    with pd.ExcelWriter(outputPath, mode="a", if_sheet_exists='replace') as writer:
        output_table.to_excel(writer, sheet_name = fileNum, index=False)

def troubleshooting_export():
        with pd.ExcelWriter(troubleshootingPath, mode="a", if_sheet_exists = 'new') as writer:
            drop1.to_excel(writer, sheet_name = str(n), index = False)
            qdyntable.to_excel(writer, sheet_name = "qdyn", index = False)

Run Analysis of specified folder

In [39]:
stored_data = pd.DataFrame(columns=["File", 'Drops', 'Recording Time'])

file_name_list = []
n_drops_list = []
record_time_list = []

file_list = os.listdir(binFolderPath)
n = 0

for filename in file_list:
    if filename.endswith('.bin'):
        n = n+1
        print("on file", n, "of", len(file_list), ":", filename)
        clear_output(wait = True)
        readfile(filename)
        accPick()
        locate_drops()
        if nDrops_trimmed > 0:
            file_name_list.append(filename)
            n_drops_list.append(nDrops_trimmed)
            path  =  os.path.join(binFolderPath, filename)
            c_time  = os.path.getctime(path)
            local_time = time.ctime(c_time)
            record_time_list.append(local_time)
            peakplot(x, peaks)

stored_data["File"] = file_name_list
stored_data["Drops"] = n_drops_list
stored_data["Recording Time"] = record_time_list

print(stored_data)
stored_data.to_csv(outputPath, header=None, index=None, sep=' ', mode='a')


        

            File  Drops            Recording Time
0   bLog01C5.bin      3  Fri Sep 29 17:11:30 2023
1   bLog01D3.bin      4  Fri Sep 29 17:11:33 2023
2   bLog01DA.bin      1  Fri Sep 29 17:11:37 2023
3   bLog01DB.bin      3  Fri Sep 29 17:11:36 2023
4   bLog01E0.bin      2  Fri Sep 29 17:11:45 2023
..           ...    ...                       ...
60  bLog02C1.bin      6  Fri Sep 29 17:11:55 2023
61  bLog02C2.bin      1  Fri Sep 29 17:11:55 2023
62  bLog02C3.bin      1  Fri Sep 29 17:11:55 2023
63  bLog02E3.bin      2  Fri Sep 29 17:12:07 2023
64  bLog0305.bin      2  Fri Sep 29 17:13:48 2023

[65 rows x 3 columns]


PermissionError: [Errno 13] Permission denied: 'C:/Users/elise/Desktop/Test data - BD3 3.16.23 -Drops Only'

In [41]:
stored_data.to_csv(outputPath+"/stored_data.csv", header=None, index=None, sep=' ', mode='a')
print("Number of Drops Found:", sum(stored_data[""]))